In [1]:
import numpy as np
import pandas as pd
import os
from deepmd.infer import DeepPot
from glob import glob
from mlptools.io.read import read_from_dp_data, read_from_lmp_dump
from mlptools.descriptor.dp import get_descriptor_vector, get_predictions

import matplotlib.pyplot as plt
import seaborn as sns

Instructions for updating:
non-resource variables are not supported in the long term


/home/y1u0d2/anaconda3/envs/deepmd/lib/python3.10/importlib/__init__.py:169: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  _bootstrap._exec(spec, module)
/home/y1u0d2/anaconda3/envs/deepmd/lib/python3.10/site-packages/ovito/extensions/averaging.py:2: UserWarning: Did you accidentally install the OVITO package from the PyPI repository in an Anaconda/Miniconda Python interpreter using the 'pip' command? Note that this will likely lead to conflicts with existing libraries in the Anaconda environment, and loading of the OVITO module may subsequently fail with an error related to the Qt framework. In such a case, please uninstall the OVITO pip package first by running 'pip uninstall -y ovito PySide2 PySide6' and then install the OVITO for Anaconda package using the correct command: 

    conda install --strict-channel-priority -c https://conda.ovito.org -c conda-forge ovito

Visit https://w

In [2]:
path2dump = '/home/y1u0d2/result/lammps/scripts/Si/sputtering/deepmd/model30/0deg_100eV/dump.lammpstrj'
path2model = '/home/y1u0d2/result/deepmd/Si/30/graph.pb'
path2data = '/home/y1u0d2/result/deepmd/Si/30/data'

In [3]:
model = DeepPot(path2model)

2023-02-28 20:37:10.265291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 20:37:10.454219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 20:37:10.454531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 20:37:10.490287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [4]:
all_descriptor = np.load(os.path.join(path2data, 'descriptor.npy'))

descriptor_df = pd.DataFrame(data=all_descriptor, columns=[f'dp_{i}' for i in range(all_descriptor.shape[1])])
descriptor_df['structure_id'] = np.load(os.path.join(path2data, 'structure_id.npy'))

## Descriptor from LAMMPS dumpfile

In [5]:
allatoms_lmp = read_from_lmp_dump(path2dump)

Number of frames: 296


In [6]:
# %matplotlib widget

from mpl_toolkits.mplot3d import Axes3D

def plot_extrapolation_atoms(atoms, extra_atoms_idx, path2save, save_filename):
    fig = plt.figure(figsize = (10, 8))
    ax = fig.add_subplot(111, projection='3d')

    ax.set_title(save_filename, size = 20)

    ax.set_xlabel("x", size = 14, color = "r")
    ax.set_ylabel("y", size = 14, color = "r")
    ax.set_zlabel("z", size = 14, color = "r")

    # 軸目盛を設定
    # ax.set_xticks([-5.0, -2.5, 0.0, 2.5, 5.0])
    # ax.set_yticks([-5.0, -2.5, 0.0, 2.5, 5.0])
    ax.set_xlim(0, atoms.cell[0].max())
    ax.set_ylim(0, atoms.cell[1].max())
    ax.set_zlim(0, atoms.cell[2].max())

    x = atoms.coord[:, 0]
    y = atoms.coord[:, 1]
    z = atoms.coord[:, 2]
    extra_atoms_coord = atoms.coord[extra_atoms_idx]

    ax.scatter(x, y, z, s=20, c = "blue", alpha=0.2)
    ax.scatter(extra_atoms_coord[:, 0], extra_atoms_coord[:, 1], extra_atoms_coord[:, 2], s=20, c = "red", alpha=1.0)
    ax.view_init(elev=10, azim=20)

    # plt.show()
    fig.savefig(os.path.join(path2save, save_filename))
    plt.close(fig)

In [7]:
scf_descriptor_df = descriptor_df.drop(columns=['structure_id'])

max_val = scf_descriptor_df.max(axis=0)
min_val = scf_descriptor_df.min(axis=0)
minmax_df = pd.concat([max_val.rename('max'), min_val.rename('min')], axis=1)

In [9]:
extra_descriptor = []
n_frames = allatoms_lmp[-1].frame
path2save = os.path.join(path2data, 'img')

for atoms in allatoms_lmp:
    print(f'frame: {atoms.frame} / {n_frames}, n_atoms: {atoms.n_atoms}')
    max_mat = np.tile(max_val.values, (atoms.n_atoms, 1))
    min_mat = np.tile(min_val.values, (atoms.n_atoms, 1))
    
    descriptor = get_descriptor_vector(atoms, model=model)
    descriptor = descriptor[0]
    rows, cols = np.where((descriptor < min_mat) | (descriptor > max_mat) == True)

    print(f'number of extrapolation descriptors: {len(list(set(rows)))} / {atoms.n_atoms}')
    print('-'*50)
    extra_descriptor.append(descriptor[sorted(list(set(rows)))])

    save_filename = f'coord{atoms.frame}'
    plot_extrapolation_atoms(
        atoms=atoms,
        extra_atoms_idx=list(set(rows)),
        path2save=path2save,
        save_filename=save_filename
    )
    
    # break

frame: 0 / 295, n_atoms: 6750
number of extrapolation descriptors: 29 / 6750
--------------------------------------------------
frame: 1 / 295, n_atoms: 6751
number of extrapolation descriptors: 29 / 6751
--------------------------------------------------
frame: 2 / 295, n_atoms: 6751
number of extrapolation descriptors: 29 / 6751
--------------------------------------------------
frame: 3 / 295, n_atoms: 6751
number of extrapolation descriptors: 30 / 6751
--------------------------------------------------
frame: 4 / 295, n_atoms: 6751
number of extrapolation descriptors: 31 / 6751
--------------------------------------------------
frame: 5 / 295, n_atoms: 6751
number of extrapolation descriptors: 29 / 6751
--------------------------------------------------
frame: 6 / 295, n_atoms: 6751
number of extrapolation descriptors: 30 / 6751
--------------------------------------------------
frame: 7 / 295, n_atoms: 6751
number of extrapolation descriptors: 29 / 6751
---------------------------

In [25]:
from PIL import Image

def make_gif(path2save):
    frames = [Image.open(image) for image in glob(f"{path2save}/*.png")]
    frame_one = frames[0]
    frame_one.save(os.path.join(path2data, "descriptor_analysis_during_sputtering.gif"), format="GIF", append_images=frames,
               save_all=True, duration=200, loop=0)
    
make_gif(path2save)


In [ ]:
# %matplotlib ipympl

# from matplotlib import pyplot as plt
# import numpy as np
# import mpl_toolkits.mplot3d.axes3d as p3
# from matplotlib import animation

# fig = plt.figure()
# ax = p3.Axes3D(fig)

# def gen(n):
#     phi = 0
#     while phi < 2*np.pi:
#         yield np.array([np.cos(phi), np.sin(phi), phi])
#         phi += 2*np.pi/n

# def update(num, data, line):
#     line.set_data(data[:2, :num])
#     line.set_3d_properties(data[2, :num])

# N = 100
# data = np.array(list(gen(N))).T
# line, = ax.plot(data[0, 0:1], data[1, 0:1], data[2, 0:1])
# # Setting the axes properties
# ax.set_xlim3d([-1.0, 1.0])
# ax.set_xlabel('X')
# ax.set_ylim3d([-1.0, 1.0])
# ax.set_ylabel('Y')
# ax.set_zlim3d([0.0, 10.0])
# ax.set_zlabel('Z')
# ani = animation.FuncAnimation(fig, update, N, fargs=(data, line), interval=10000/N, blit=False)
# #ani.save('matplot003.gif', writer='imagemagick')
# plt.show()